In [1]:
import os
from pprint import pprint
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient


In [2]:
# Subscription id can be found at https://portal.azure.com/#blade/Microsoft_Azure_Billing/SubscriptionsBlade
subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID') 

# Register an Application to get the client id: https://portal.azure.com/#blade/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/RegisteredApps
# Add a Key to the registered App to get the CLIENT_SECRET
# Tenant id is the same as the directory id, and can be found in the AD->properties page
# Where the heck to I configure rbac for this application?
# OK, shortcut, there is a cli commmand that will create an application and configure RBAC:
# az ad sp create-for-rbac
# I can't find where applications are on the UI, but this does work... 
credentials = ServicePrincipalCredentials(
    client_id=os.environ['AZURE_CLIENT_ID'],
    secret=os.environ['AZURE_CLIENT_SECRET'],
    tenant=os.environ['AZURE_TENANT_ID']
)

resource_client = ResourceManagementClient(credentials, subscription_id)
compute_client = ComputeManagementClient(credentials, subscription_id)

In [22]:
# resource_client.resources does not include resource group info 
# which we need to get to the vm, so we have to start by getting all the groups
for groups in resource_client.resource_groups.list(): 

    resources = resource_client.resources.list_by_resource_group(groups.name)
    for resource in resources:
        if resource.type.startswith("Microsoft.Compute/virtualMachines"):
            # The only way to get the status of the VM is by looking at the instance View :(
            # and the only way to see the instance view is by looking at the vms 1 by 1, instead of 
            # calling vm.list_all() or vm.list()
            vm = compute_client.virtual_machines.get(
                resource_group_name=groups.name,
                vm_name=resource.name, expand='instanceView')
            
            # Interesting things on this instance view: 
            #    statuses-> what's the status on this thing?
            #    os_name -> what's the os
            pprint(vars(vm.instance_view))

            # There are multiple statuses because azure has a `ProvisioningState/succeeded` status
            # When the VM is shutdown, the `PowerState/running` status disappears
            for status in vm.instance_view.statuses:
                if status.code == "PowerState/running":               
                    pprint("{} is running!".format(resource.name))    

{'additional_properties': {},
 'boot_diagnostics': <azure.mgmt.compute.v2018_06_01.models.boot_diagnostics_instance_view_py3.BootDiagnosticsInstanceView object at 0x7f8aceecd400>,
 'computer_name': 'test-vm-1',
 'disks': [<azure.mgmt.compute.v2018_06_01.models.disk_instance_view_py3.DiskInstanceView object at 0x7f8aceecd5f8>],
 'extensions': None,
 'maintenance_redeploy_status': None,
 'os_name': 'centos',
 'os_version': '7.5.1804',
 'platform_fault_domain': None,
 'platform_update_domain': None,
 'rdp_thumb_print': None,
 'statuses': [<azure.mgmt.compute.v2018_06_01.models.instance_view_status_py3.InstanceViewStatus object at 0x7f8aceecd940>,
 'vm_agent': <azure.mgmt.compute.v2018_06_01.models.virtual_machine_agent_instance_view_py3.VirtualMachineAgentInstanceView object at 0x7f8aceecddd8>}
'test-vm-1 is running!'
